In [1]:
include("lib/all.jl");

In [2]:
test_mov()

705833

In [3]:
test_acc()

839855

In [4]:
particles = make_particles(2^20);
t = Tree(particles, 12);

In [26]:
@time group!(t);

  0.081803 seconds (4 allocations: 160 bytes)


In [27]:
@time inform!(t);

  0.068685 seconds (1.10 M allocations: 16.856 MB, 1.68% gc time)


In [29]:
test_exp();

LoadError: MethodError: no method matching Node(::Float64, ::Float64, ::Float64, ::Float64, ::Float64, ::Float64, ::Float64, ::Float64, ::Float64, ::Float64, ::Float64, ::Int64, ::Int64, ::Int64, ::Int64, ::Int64, ::Int64)[0m
Closest candidates are:
  Node(::Any, ::Any, ::Any, ::Any, ::Any, ::Any, ::Any, ::Any, ::Any, ::Any) at /home/skariel/Desktop/nbtree/lib/tree.jl:37
  Node(::Float64, ::Float64, ::Float64, ::Float64, ::Float64, [1m[31m::Int64[0m, [1m[31m::Int64[0m, [1m[31m::Int64[0m, [1m[31m::Int64[0m, [1m[31m::Int64[0m) at /home/skariel/Desktop/nbtree/lib/tree.jl:37
  Node{T}(::Any) at sysimg.jl:53[0m

In [30]:
test_in_cell_mass(t)

LoadError: type Node has no field maxx

In [31]:
ax = zeros(length(particles))
ay = zeros(length(particles))
az = zeros(length(particles));

In [32]:
@time interact!(t, 0.5, ax, ay, az);

  1.488059 seconds (4 allocations: 160 bytes)


In [33]:
@time collect!(t, ax,ay,az)

  0.141525 seconds (8.30 M allocations: 126.664 MB, 48.92% gc time)


In [34]:
@time deliver!(t, ax,ay,az)

  0.209391 seconds (31.46 M allocations: 480.000 MB, 7.85% gc time)


In [32]:
perf(t.particles, ax, 1000, 1.0e-178)

1.1641629853468272